In [114]:
import requests
import time
import certifi
import pandas as pd

base_url = "https://api.scryfall.com/"

SET = 'dft'
PRICE = 2
search_query = f"s:{SET} usd<{PRICE} unique:prints -type:basic r>u -is:reprint"
search_url = f"{base_url}cards/search?q={search_query}"
headers = {
        "User-Agent": "scyrfall_tools/1.0 (jackson.travis.do@gmail.com)"  # Replace with your app name and email
    }

card_list = pd.DataFrame({'name':[],
                   'price':[], 
                   'fullart':[], 
                   'frame':[], 
                   'frame_effects': [], 
                   'border_color':[]
                   })

while search_url:
    response = requests.get(search_url, headers=headers, verify=False)
    if response.status_code == 200:
        data = response.json()
        
        # Print the names of the cards found in this page
        for card in data['data']:
            print(f"{card['name']} \t\t\t {card['prices']['usd']}")
            #for key in card.keys():
            #    if key not in df.columns:
            #        df[key] = None
            entry = {'name':card['name'], 'price':card['prices']['usd'], 'fullart':card['full_art'], 'frame':card['frame'], 'border_color':card['border_color']}
            if 'frame_effects' in card.keys():
                entry['frame_effects'] = card['frame_effects']
            card_list = pd.concat([card_list, pd.DataFrame([entry])], ignore_index=True)
        search_url = data.get('next_page', None)
        time.sleep(0.1)
    else:
        print("Error:", response.status_code, response.json().get("details"))





c:\src\scryfall-tools\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scryfall.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Aatchik, Emerald Radian 			 0.23
Aatchik, Emerald Radian 			 0.58
Afterburner Expert 			 0.44
Afterburner Expert 			 0.46
Agonasaur Rex 			 1.07
Agonasaur Rex 			 1.08
Basri, Tomorrow's Champion 			 0.24
Basri, Tomorrow's Champion 			 0.28
Boommobile 			 0.17
Boommobile 			 0.20
Boommobile 			 None
Bulwark Ox 			 0.25
Bulwark Ox 			 0.21
Bulwark Ox 			 None
Burnout Bashtronaut 			 0.35
Burnout Bashtronaut 			 0.39
Captain Howler, Sea Scourge 			 0.38
Captain Howler, Sea Scourge 			 1.05
Caradora, Heart of Alacria 			 0.18
Caradora, Heart of Alacria 			 0.19
Coalstoke Gearhulk 			 0.91
Coalstoke Gearhulk 			 1.73
Count on Luck 			 0.36
Count on Luck 			 0.26
Cryptcaller Chariot 			 0.57
Cryptcaller Chariot 			 0.99
Cursecloth Wrappings 			 0.34
Cursecloth Wrappings 			 0.46
Daretti, Rocketeer Engineer 			 0.19
Daretti, Rocketeer Engineer 			 0.28
Debris Beetle 			 0.19
Debris Beetle 			 0.28
Demonic Junker 			 0.39
Demonic Junker 			 0.45
District Mascot 			 0.47
District Mascot 			 0.5

In [128]:
# Generate list for TCGplayer
import numpy as np

def get_tcgplayer_name(card):
    entry = card['name']
    if type(card['frame_effects']) == list:
        if 'showcase' in card['frame_effects']:
            entry = f"{entry} (Showcase)"
        if card['frame_effects'] == ['extendedart']:
            entry = f"{entry} (Extended)"
    if card['frame'] == '1997':
        entry = f"{entry} (Retro)"
    if card['border_color'] == 'borderless':
        entry = f"{entry} (Borderless)"
    return entry

buylist = pd.DataFrame()

for name in card_list['name'].unique():
    cards = card_list[card_list['name']==name]
    for index,card in cards.iterrows():
        entry = get_tcgplayer_name(card)
        buylist = pd.concat([buylist, pd.DataFrame([{'Quantity': 1, 'Name':entry, 'Price':card['price']}])], ignore_index=True)

    if len(cards) < 3:
        # Find cheapest printing
        to_add = cards[cards['name']==name].loc[cards['price'].idxmin()]
        buylist.loc[buylist['Name']==get_tcgplayer_name(to_add), 'Quantity'] += 1
        buylist = buylist.sort_values(by="Name")

buylist.to_csv('test.csv')

PermissionError: [Errno 13] Permission denied: 'test.csv'

In [101]:
buylist['Total Price'] = buylist['Quantity'].astype(int)*buylist['Price'].astype(float)
buylist['Total Price'].sum()

np.float64(98.31)

In [78]:
len(buylist[buylist['Name']=='Zealous Conscripts'])

1

In [13]:
card['foil']

True

# Testing refactoring

In [6]:
import requests
import time
import certifi
import pandas as pd

class ScryfallCardFetcher:
    """Class to query Scryfall API and fetch card data based on search criteria."""

    BASE_URL = "https://api.scryfall.com/"
    HEADERS = {
        "User-Agent": "scyrfall_tools/1.0 (jackson.travis.do@gmail.com)"
    }

    def __init__(self, set_code: str, max_price: float, max_retries: int = 3):
        self.set_code = set_code
        self.max_price = max_price
        self.max_retries = max_retries
        self.card_list = pd.DataFrame({'name': [], 'price': [], 'fullart': [], 'frame': [], 'frame_effects': [], 'border_color': []})

    def build_query(self) -> str:
        """Constructs the search query for Scryfall API."""
        return f"s:{self.set_code} usd<{self.max_price} unique:prints -type:basic r>u"

    def fetch_cards(self):
        """Fetches card data using the constructed query with error handling and retries."""
        search_url = f"{self.BASE_URL}cards/search?q={self.build_query()}"
        while search_url:
            for attempt in range(self.max_retries):
                try:
                    response = requests.get(search_url, headers=self.HEADERS, verify=False)
                    response.raise_for_status()
                    data = response.json()
                    self.process_cards(data['data'])
                    search_url = data.get('next_page', None)
                    time.sleep(0.1)
                    break
                except requests.exceptions.RequestException as e:
                    print(f"Attempt {attempt + 1} failed: {e}")
                    if attempt == self.max_retries - 1:
                        print("Max retries reached. Skipping this request.")
                        search_url = None

    def process_cards(self, cards: list):
        """Processes and stores card data into a DataFrame."""
        for card in cards:
            #print(f"{card['name']} \t\t\t {card['prices']['usd']}")
            entry = {
                'name': card['name'],
                'price': card['prices']['usd'],
                'fullart': card.get('full_art', False),
                'frame': card.get('frame', ''),
                'border_color': card.get('border_color', '')
            }
            if 'frame_effects' in card:
                entry['frame_effects'] = card['frame_effects']
            self.card_list = pd.concat([self.card_list, pd.DataFrame([entry])], ignore_index=True)

    def get_card_data(self) -> pd.DataFrame:
        """Returns the collected card data as a DataFrame."""
        return self.card_list

# Example usage
if __name__ == "__main__":
    fetcher = ScryfallCardFetcher(set_code='dft', max_price=200)
    fetcher.fetch_cards()
    print(fetcher.get_card_data())

d:\src\scryfall-tools\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scryfall.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\src\scryfall-tools\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scryfall.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


                        name price fullart frame          frame_effects  \
0    Aatchik, Emerald Radian  0.27     0.0  2015            [legendary]   
1    Aatchik, Emerald Radian  0.59   False  2015  [legendary, inverted]   
2    Aatchik, Emerald Radian  None   False  2015  [legendary, inverted]   
3         Afterburner Expert  0.36   False  2015                    NaN   
4         Afterburner Expert  0.47   False  2015          [extendedart]   
..                       ...   ...     ...   ...                    ...   
291     Winter, Cursed Rider  0.23   False  2015  [legendary, inverted]   
292     Winter, Cursed Rider  None   False  2015  [legendary, inverted]   
293      Zahur, Glory's Past  0.25   False  2015            [legendary]   
294      Zahur, Glory's Past  0.55   False  2015  [legendary, inverted]   
295      Zahur, Glory's Past  None   False  2015  [legendary, inverted]   

    border_color  
0          black  
1     borderless  
2         yellow  
3          black  
4   

In [21]:
import requests
import time
import certifi
import pandas as pd

class ScryfallCardFetcher:
    """Class to query Scryfall API and fetch card data based on search criteria."""

    BASE_URL = "https://api.scryfall.com/"
    HEADERS = {
        "User-Agent": "scyrfall_tools/1.0 (jackson.travis.do@gmail.com)"
    }

    def __init__(self, set_code: str, max_price: float, max_retries: int = 3, exclude_special: list = None):
        self.set_code = set_code
        self.max_price = max_price
        self.max_retries = max_retries
        self.exclude_special = exclude_special or []
        self.card_list = pd.DataFrame({'name': [], 'price': [], 'fullart': [], 'frame': [], 'frame_effects': [], 'border_color': []})
        self.buylist = pd.DataFrame()

    def build_query(self) -> str:
        """Constructs the search query for Scryfall API."""
        return f"s:{self.set_code} usd<{self.max_price} unique:prints -type:basic r>u"

    def fetch_cards(self):
        """Fetches card data using the constructed query with error handling and retries."""
        search_url = f"{self.BASE_URL}cards/search?q={self.build_query()}"
        while search_url:
            for attempt in range(self.max_retries):
                try:
                    response = requests.get(search_url, headers=self.HEADERS, verify=False)
                    response.raise_for_status()
                    data = response.json()
                    self.process_cards(data['data'])
                    search_url = data.get('next_page', None)
                    time.sleep(0.1)
                    break
                except requests.exceptions.RequestException as e:
                    print(f"Attempt {attempt + 1} failed: {e}")
                    if attempt == self.max_retries - 1:
                        print("Max retries reached. Skipping this request.")
                        search_url = None

    def process_cards(self, cards: list):
        """Processes and stores card data into a DataFrame."""
        for card in cards:
            frame_effects = card.get('frame_effects', [])
            frame_effects = frame_effects if isinstance(frame_effects, list) else [frame_effects]

            if any(effect in self.exclude_special for effect in frame_effects):
                continue

            print(f"{card['name']} \t\t\t {card['prices']['usd']}")
            entry = {
                'name': card['name'],
                'price': card['prices']['usd'],
                'fullart': card.get('full_art', False),
                'frame': card.get('frame', ''),
                'border_color': card.get('border_color', ''),
                'frame_effects': frame_effects
            }
            self.card_list = pd.concat([self.card_list, pd.DataFrame([entry])], ignore_index=True)

    def get_tcgplayer_name(self, card):
        """Generate TCGplayer-friendly card names based on frame, effects, and border."""
        entry = card['name']
        if 'showcase' in card.get('frame_effects', []):
            entry = f"{entry} (Showcase)"
        if card.get('frame') == '1997':
            entry = f"{entry} (Retro)"
        if card.get('border_color') == 'borderless':
            entry = f"{entry} (Borderless)"
        return entry

    def generate_buylist(self):
        """Generate a buylist for TCGplayer based on fetched cards."""
        for name in self.card_list['name'].unique():
            cards = self.card_list[self.card_list['name'] == name]
            for _, card in cards.iterrows():
                entry = self.get_tcgplayer_name(card)
                self.buylist = pd.concat([self.buylist, pd.DataFrame([{'Quantity': 1, 'Name': entry, 'Price': card['price']}])], ignore_index=True)

            if len(cards) < 3:
                to_add = cards.loc[cards['price'].idxmin()]
                self.buylist.loc[self.buylist['Name'] == self.get_tcgplayer_name(to_add), 'Quantity'] += 1

        self.buylist = self.buylist.sort_values(by="Name")

    def get_card_data(self) -> pd.DataFrame:
        """Returns the collected card data as a DataFrame."""
        return self.card_list

    def get_buylist(self) -> pd.DataFrame:
        """Returns the generated buylist as a DataFrame."""
        return self.buylist

# Example usage
if __name__ == "__main__":
    fetcher = ScryfallCardFetcher(set_code='dft', max_price=2, exclude_special=['showcase', 'retro', 'borderless'])
    fetcher.fetch_cards()
    fetcher.generate_buylist()
    print(fetcher.get_card_data())
    print(fetcher.get_buylist())


d:\src\scryfall-tools\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scryfall.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Aatchik, Emerald Radian 			 0.27
Aatchik, Emerald Radian 			 0.59
Afterburner Expert 			 0.36
Afterburner Expert 			 0.47
Agonasaur Rex 			 0.86
Agonasaur Rex 			 1.11
Basri, Tomorrow's Champion 			 0.24
Basri, Tomorrow's Champion 			 0.29
Bloodghast 			 0.80
Bloodghast 			 1.15
Boommobile 			 0.17
Boommobile 			 0.21
Boommobile 			 None
Bulwark Ox 			 0.26
Bulwark Ox 			 0.39
Bulwark Ox 			 None
Burnout Bashtronaut 			 0.35
Burnout Bashtronaut 			 0.40
Captain Howler, Sea Scourge 			 0.37
Captain Howler, Sea Scourge 			 1.03
Caradora, Heart of Alacria 			 0.18
Caradora, Heart of Alacria 			 0.19
Coalstoke Gearhulk 			 0.82
Coalstoke Gearhulk 			 1.71
Count on Luck 			 0.33
Count on Luck 			 0.25
Cryptcaller Chariot 			 0.52
Cryptcaller Chariot 			 0.84
Cursecloth Wrappings 			 0.33
Cursecloth Wrappings 			 0.44
Daretti, Rocketeer Engineer 			 0.20
Daretti, Rocketeer Engineer 			 0.28
Debris Beetle 			 0.17
Debris Beetle 			 0.28
Demonic Junker 			 0.37
Demonic Junker 			 0.42
District

In [22]:
fetcher.get_buylist()

,Quantity,Name,Price
0,2,"Aatchik, Emerald Radian",0.27
1,1,"Aatchik, Emerald Radian (Borderless)",0.59
2,2,Afterburner Expert,0.36
3,2,Afterburner Expert,0.47
4,2,Agonasaur Rex,0.86
...,...,...,...
123,1,Webstrike Elite (Borderless),0.28
124,2,"Winter, Cursed Rider",0.16
125,1,"Winter, Cursed Rider (Borderless)",0.23
126,2,"Zahur, Glory's Past",0.25


In [ ]:
import requests
import time
import certifi
import pandas as pd

class ScryfallCardFetcher:
    """Class to query Scryfall API and fetch card data based on search criteria."""

    BASE_URL = "https://api.scryfall.com/"
    HEADERS = {
        "User-Agent": "scyrfall_tools/1.0 (jackson.travis.do@gmail.com)"
    }
    VALID_SPECIALS = {'showcase', 'retro', 'borderless', 'extendedart'}

    def __init__(self, set_code: str, max_price: float, max_retries: int = 3, exclude_reprints: bool = True, exclude_special: list = None):
        self.set_code = set_code
        self.max_price = max_price
        self.max_retries = max_retries
        self.exclude_special = [s for s in (exclude_special or []) if s in self.VALID_SPECIALS]
        self.exclude_reprints = exclude_reprints
        self.card_list = pd.DataFrame({'name': [], 'price': [], 'fullart': [], 'frame': [], 'frame_effects': [], 'border_color': []})
        self.buylist = pd.DataFrame()

    def build_query(self) -> str:
        """Constructs the search query for Scryfall API."""
        query = f"s:{self.set_code} usd<{self.max_price} unique:prints -type:basic r>u"
        if self.exclude_reprints:
            query = query + '-is:reprint'
        return query

    def fetch_cards(self):
        """Fetches card data using the constructed query with error handling and retries."""
        search_url = f"{self.BASE_URL}cards/search?q={self.build_query()}"
        while search_url:
            for attempt in range(self.max_retries):
                try:
                    response = requests.get(search_url, headers=self.HEADERS, verify=False)
                    response.raise_for_status()
                    data = response.json()
                    self.process_cards(data['data'])
                    search_url = data.get('next_page', None)
                    time.sleep(0.1)
                    break
                except requests.exceptions.RequestException as e:
                    print(f"Attempt {attempt + 1} failed: {e}")
                    if attempt == self.max_retries - 1:
                        print("Max retries reached. Skipping this request.")
                        search_url = None

    def process_cards(self, cards: list):
        """Processes and stores card data into a DataFrame."""
        for card in cards:
            frame_effects = card.get('frame_effects', [])
            frame_effects = frame_effects if isinstance(frame_effects, list) else [frame_effects]
            retro_effects = card.get('frame', [])
            retro_effects = retro_effects if isinstance(retro_effects, list) else [retro_effects]
            border_effects = card.get('border_color', [])
            border_effects = border_effects if isinstance(border_effects, list) else [border_effects]

            alternate_arts = frame_effects + retro_effects + border_effects

            if any(effect.replace('1997', 'retro') in self.exclude_special for effect in alternate_arts):
                continue

            entry = {
                'name': card['name'],
                'price': card['prices']['usd'],
                'fullart': card.get('full_art', False),
                'frame': card.get('frame', ''),
                'border_color': card.get('border_color', ''),
                'frame_effects': frame_effects
            }
            self.card_list = pd.concat([self.card_list, pd.DataFrame([entry])], ignore_index=True)

    def get_tcgplayer_name(self, card):
        """Generate TCGplayer-friendly card names based on frame, effects, and border."""
        entry = card['name']
        if 'showcase' in card.get('frame_effects', []):
            entry = f"{entry} (Showcase)"
        if 'extendedart' in card.get('frame_effects', []):
            entry = f"{entry} (Extended Art)"
        if '1997' in card.get('frame'):
            entry = f"{entry} (Retro)"
        if 'borderless' in card.get('border_color'):
            entry = f"{entry} (Borderless)"
        return entry

    def generate_buylist(self):
        """Generate a buylist for TCGplayer based on fetched cards."""
        for name in self.card_list['name'].unique():
            cards = self.card_list[self.card_list['name'] == name]
            for _, card in cards.iterrows():
                entry = self.get_tcgplayer_name(card)
                self.buylist = pd.concat([self.buylist, pd.DataFrame([{'Quantity': 1, 'Name': entry, 'Price': card['price']}])], ignore_index=True)

            while self.buylist[self.buylist['Name'].str.contains(name)]['Quantity'].sum() < 3:
                to_add = cards.loc[cards['price'].dropna().idxmin()]
                self.buylist.loc[self.buylist['Name'] == self.get_tcgplayer_name(to_add), 'Quantity'] += 1

        self.buylist = self.buylist.sort_values(by="Name")

    def get_card_data(self) -> pd.DataFrame:
        """Returns the collected card data as a DataFrame."""
        return self.card_list

    def get_buylist(self) -> pd.DataFrame:
        """Returns the generated buylist as a DataFrame."""
        return self.buylist

# Example usage
if __name__ == "__main__":
    fetcher = ScryfallCardFetcher(set_code='dft', max_price=2, exclude_special=['showcase', 'retro'])
    fetcher.fetch_cards()
    fetcher.generate_buylist()
    #fetcher.get_card_data())
    fetcher.get_buylist()


d:\src\scryfall-tools\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scryfall.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Aatchik, Emerald Radian 			 0.27
Aatchik, Emerald Radian 			 0.59
Afterburner Expert 			 0.36
Afterburner Expert 			 0.47
Agonasaur Rex 			 0.86
Agonasaur Rex 			 1.11
Basri, Tomorrow's Champion 			 0.24
Basri, Tomorrow's Champion 			 0.29
Bloodghast 			 0.80
Bloodghast 			 1.15
Boommobile 			 0.17
Boommobile 			 0.21
Boommobile 			 None
Bulwark Ox 			 0.26
Bulwark Ox 			 0.39
Bulwark Ox 			 None
Burnout Bashtronaut 			 0.35
Burnout Bashtronaut 			 0.40
Captain Howler, Sea Scourge 			 0.37
Captain Howler, Sea Scourge 			 1.03
Caradora, Heart of Alacria 			 0.18
Caradora, Heart of Alacria 			 0.19
Coalstoke Gearhulk 			 0.82
Coalstoke Gearhulk 			 1.71
Count on Luck 			 0.33
Count on Luck 			 0.25
Cryptcaller Chariot 			 0.52
Cryptcaller Chariot 			 0.84
Cursecloth Wrappings 			 0.33
Cursecloth Wrappings 			 0.44
Daretti, Rocketeer Engineer 			 0.20
Daretti, Rocketeer Engineer 			 0.28
Debris Beetle 			 0.17
Debris Beetle 			 0.28
Demonic Junker 			 0.37
Demonic Junker 			 0.42
District

In [84]:
fetcher = ScryfallCardFetcher(set_code='dft', max_price=2, exclude_special=['showcase', 'retro', 'borderless', 'extendedart'])
fetcher.fetch_cards()
fetcher.generate_buylist()
#fetcher.get_card_data())
fetcher.get_buylist()['Quantity'].sum()

d:\src\scryfall-tools\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scryfall.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


np.int64(186)

In [9]:
import requests
import time
import certifi
import pandas as pd

class CardList:
    def __init__(self):
        self.data = pd.DataFrame({'name': [], 'price': [], 'fullart': [], 'frame': [], 'frame_effects': [], 'border_color': []})

    def add_card(self, card):
        self.data = pd.concat([self.data, pd.DataFrame([card])], ignore_index=True)

    def get_data(self):
        return self.data

class Buylist:
    def __init__(self):
        self.data = pd.DataFrame()

    def add_entry(self, name, price):
        self.data = pd.concat([self.data, pd.DataFrame([{'Quantity': 1, 'Name': name, 'Price': price}])], ignore_index=True)

    def adjust_quantity(self, name, adjust: int = 1):
        self.data.loc[self.data['Name'] == name, 'Quantity'] += adjust
        if self.data.loc[self.data['Name'] == name]['Quantity'].values < 1:
            self.data.drop(self.data.loc[self.data['Name'] == name].index, inplace = True)

    def sort_buylist(self):
        self.data = self.data.sort_values(by="Name")

    def export_to_excel(self, filename='buylist.xlsx'):
        self.data.to_excel(filename, index=False)

    def get_data(self):
        return self.data

class ScryfallCardFetcher:
    BASE_URL = "https://api.scryfall.com/"
    HEADERS = {"User-Agent": "scyrfall_tools/1.0 (jackson.travis.do@gmail.com)"}
    VALID_SPECIALS = {'showcase', 'retro', 'borderless', 'extendedart'}

    def __init__(self, set_code: str, max_price: float, copies: int = 4, max_retries: int = 3, exclude_reprints: bool = True, exclude_special: list = None):
        self.set_code = set_code
        self.max_price = max_price
        self.copies = copies
        self.max_retries = max_retries
        self.exclude_special = [s for s in (exclude_special or []) if s in self.VALID_SPECIALS]
        self.exclude_reprints = exclude_reprints
        self.card_list = CardList()
        self.buylist = Buylist()

    def build_query(self) -> str:
        query = f"s:{self.set_code} usd<{self.max_price} unique:prints -type:basic r>u"
        if self.exclude_reprints:
            query = query + ' -is:reprint'
        return query

    def fetch_cards(self):
        search_url = f"{self.BASE_URL}cards/search?q={self.build_query()}"
        while search_url:
            for attempt in range(self.max_retries):
                try:
                    response = requests.get(search_url, headers=self.HEADERS, verify=False)
                    response.raise_for_status()
                    data = response.json()
                    self.process_cards(data['data'])
                    search_url = data.get('next_page', None)
                    time.sleep(0.1)
                    break
                except requests.exceptions.RequestException as e:
                    print(f"Attempt {attempt + 1} failed: {e}")
                    if attempt == self.max_retries - 1:
                        print("Max retries reached. Skipping this request.")
                        search_url = None

    def process_cards(self, cards: list):
        for card in cards:
            frame_effects = card.get('frame_effects', [])
            frame_effects = frame_effects if isinstance(frame_effects, list) else [frame_effects]
            retro_effects = card.get('frame', [])
            retro_effects = retro_effects if isinstance(retro_effects, list) else [retro_effects]
            border_effects = card.get('border_color', [])
            border_effects = border_effects if isinstance(border_effects, list) else [border_effects]

            alternate_arts = frame_effects + retro_effects + border_effects

            if any(effect.replace('1997', 'retro') in self.exclude_special for effect in alternate_arts):
                continue

            entry = {
                'name': card['name'],
                'price': card['prices']['usd'],
                'fullart': card.get('full_art', False),
                'frame': card.get('frame', ''),
                'border_color': card.get('border_color', ''),
                'frame_effects': frame_effects
            }
            self.card_list.add_card(entry)

    def get_tcgplayer_name(self, card):
        entry = card['name']
        entry = entry.split(' //')[0]
        if 'showcase' in card.get('frame_effects', []):
            entry = f"{entry} (Showcase)"
        if 'extendedart' in card.get('frame_effects', []):
            entry = f"{entry} (Extended Art)"
        if '1997' in card.get('frame'):
            entry = f"{entry} (Retro Frame)"
        if 'borderless' in card.get('border_color'):
            entry = f"{entry} (Borderless)"
        return entry

    def generate_buylist(self):
        for name in self.card_list.get_data()['name'].unique():
            cards = self.card_list.get_data()[self.card_list.get_data()['name'] == name]
            for _, card in cards.iterrows():
                entry = self.get_tcgplayer_name(card)
                self.buylist.add_entry(entry, card['price'])

            while self.buylist.get_data()[self.buylist.get_data()['Name'].str.contains(name.split(' //')[0])]['Quantity'].sum() > self.copies:
                current_prints = self.buylist.get_data()[self.buylist.get_data()['Name'].str.contains(name.split(' //')[0])]
                to_drop = current_prints.loc[current_prints['Price'].dropna().idxmax()]['Name']
                self.buylist.adjust_quantity(to_drop, -1)              

            while self.buylist.get_data()[self.buylist.get_data()['Name'].str.contains(name.split(' //')[0])]['Quantity'].sum() < self.copies:
                to_add = cards.loc[cards['price'].dropna().idxmin()]
                self.buylist.adjust_quantity(self.get_tcgplayer_name(to_add), 1)
                print(self.get_tcgplayer_name(to_add))

        self.buylist.sort_buylist()

# Example usage
if __name__ == "__main__":
    fetcher = ScryfallCardFetcher(set_code='inr', max_price=2, copies = 2, exclude_reprints = False)
    fetcher.fetch_cards()
    fetcher.generate_buylist()
    fetcher.buylist.export_to_excel('buylist.xlsx')


d:\src\scryfall-tools\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scryfall.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Altered Ego
Angelfire Ignition
Bloodhall Priest
Bruna, the Fading Light
Captivating Vampire (Retro Frame)
Chandra, Dressed to Kill
Collective Defiance
Conjurer's Closet (Retro Frame)
Decimator of the Provinces
Distended Mindbender
Docent of Perfection
Dreamroot Cascade
Elder Deep-Fiend
Falkenrath Gorger
Garruk Relentless
Gisa and Geralf
Grimgrin, Corpse-Born
Hanweir Battlements
Hanweir Garrison
Helvault
Invasion of Innistrad
Jace, Unraveler of Secrets
Kruin Outlaw
Maelstrom Pulse
Mayor of Avabruck
Mirrorwing Dragon
Overcharged Amalgam
Skirsdag High Priest
Stitcher's Graft
Tamiyo, Field Researcher
Tamiyo's Journal
The Gitrog Monster (Retro Frame)
Torens, Fist of the Angels
Unnatural Growth
Vanquish the Horde
Voice of the Blessed
Voldaren Bloodcaster
Wrenn and Seven


In [91]:
fetcher.buylist.get_data()

,Quantity,Name,Price
0,2,"Aatchik, Emerald Radian",0.27
1,1,"Aatchik, Emerald Radian (Borderless)",0.59
2,3,Adrenaline Jockey,0.06
4,3,Aether Syphon,0.07
3,3,Aetherjacket,0.02
...,...,...,...
346,3,Wreck Remover,0.03
345,3,Wreckage Wickerfolk,0.02
347,3,Wretched Doll,0.05
348,2,"Zahur, Glory's Past",0.25


In [105]:
buylist

,Quantity,Name,Price
0,2,Adaptive Omnitool,1.32
1,2,Adaptive Omnitool,1.12
2,2,Aetherflux Conduit,0.69
3,2,Aetherflux Conduit,0.60
4,2,Lost Monarch of Ifnir,0.96
5,2,Lost Monarch of Ifnir,0.82
6,2,"Nissa, Worldsoul Speaker",0.50
7,2,"Nissa, Worldsoul Speaker",0.53
8,2,On Wings of Gold,0.86
9,2,On Wings of Gold,0.93
